In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
from xgboost import XGBRegressor
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
preds_vs_actuals = []
lags = [0, 30, 60, 120, 180, 240, 360]
ids = ["MMCS0002", "MMCS0003", "MMCS0005", "MMCS0007", "MMCS0008", "MMCS0009", "MMCS0010", "MMCS0011", "MMCS0016"]
final_data = []
file_path = "/content/drive/MyDrive/UTA_Glucose/Ravikiran/data/feature_engineered"
results_path = "/content/drive/MyDrive/UTA_Glucose/Ravikiran/"
models = ["XGBoost"]

# Generate a timestamped file name
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create a PDF file to save all the plots
pdf_filename = f"{results_path}/glucose_predictions_plots.pdf"
pdf_pages = PdfPages(pdf_filename)

In [ ]:
def perform_kfold_cv(features, labels, model, model_name, lag, id, k_splits=5):

    features = features.reshape(features.shape[0], -1)
    labels = labels.flatten()

    kfold = KFold(n_splits=k_splits, shuffle=True, random_state=42)
    rmse_scores, mae_scores, mape_scores, r2_scores, spearman_coeffs = [], [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(features, labels)):
        print(f"Fold {fold + 1}")
        X_train, X_val = features[train_idx], features[val_idx]
        y_train, y_val = labels[train_idx], labels[val_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse_scores.append(mean_squared_error(y_val, y_pred, squared=False))
        mae_scores.append(mean_absolute_error(y_val, y_pred))
        mape_scores.append(np.mean(np.abs((y_val - y_pred) / np.where(y_val != 0, y_val, np.finfo(float).eps))) * 100)
        r2_scores.append(r2_score(y_val, y_pred))
        spearman_coeffs.append(spearmanr(y_val, y_pred)[0])
        #print(f"RMSE: {rmse_scores[-1]}, MAE: {mae_scores[-1]}, MAPE: {mape_scores[-1]}, R²: {r2_scores[-1]}, Spearman: {spearman_coeffs[-1]}")

        # Save actual and predicted values to the CSV data list
        for actual, predicted in zip(y_val, y_pred):
            preds_vs_actuals.append({
                "Model": model_name,
                "Fold": fold + 1,
                "Actual": actual,
                "Predicted": predicted,
                "Lag": lag,
                "ID": id,
            })

        # Plot Actual vs Predicted values
        plt.figure(figsize=(10, 6))
        # Plotting the actual glucose levels with a solid blue line
        plt.plot(y_test, label='Actual', linestyle='-', color='blue')

        # Plotting the predicted glucose levels with a solid red line
        plt.plot(y_pred, label='Predicted', linestyle='-', color='red')

        # Set plot title with model name and fold number
        plt.title(f"Actual vs Predicted Glucose Levels\n ID: {id}, Model: {model_name},\n Fold: {fold + 1}, Lag: {lag}", fontsize=14)
        plt.xlabel("Input Interval")
        plt.ylabel("Glucose Level")
        plt.legend()
        plt.grid(True)

        # Add text box with metrics in the plot
        plt.figtext(0.15, -0.1, f"RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}, MAE: {np.mean(mae_scores):.4f} ± {np.std(mae_scores):.4f}, \nMAPE: {np.mean(mape_scores):.4f} ± {np.std(mape_scores):.4f}, R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}, \nSpearman: {np.mean(spearman_coeffs):.4f} ± {np.std(spearman_coeffs):.4f}",
                    fontsize=10, ha='left')

        # Save the current plot to the PDF
        pdf_pages.savefig()
        plt.close()

    # Include mean and standard deviation for each metric
    metrics = {
        "RMSE Mean": np.mean(rmse_scores), "RMSE Std": np.std(rmse_scores),
        "MAE Mean": np.mean(mae_scores), "MAE Std": np.std(mae_scores),
        "MAPE Mean": np.mean(mape_scores), "MAPE Std": np.std(mape_scores),
        "R² Mean": np.mean(r2_scores), "R² Std": np.std(r2_scores),
        "Spearman Mean": np.mean(spearman_coeffs), "Spearman Std": np.std(spearman_coeffs),
        "model": model_name, "lag": lag, "id": id,
    }

    print(f"RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}, "
      f"MAE: {np.mean(mae_scores):.4f} ± {np.std(mae_scores):.4f}, "
      f"MAPE: {np.mean(mape_scores):.4f} ± {np.std(mape_scores):.4f}, "
      f"R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}, "
      f"Spearman: {np.mean(spearman_coeffs):.4f} ± {np.std(spearman_coeffs):.4f}")

    return metrics

In [ ]:
xgb_params = {
    "n_estimators": 100,               # Number of boosting rounds (trees)
    "max_depth": 6,                    # Maximum depth of each tree
    "learning_rate": 0.1,              # Learning rate (also known as eta)
    "subsample": 0.8,                  # Fraction of samples to use for training each tree
    "colsample_bytree": 0.8,           # Fraction of features to consider for each split
    "objective": "reg:squarederror",   # Objective function for regression (minimizes squared error)
    "eval_metric": "rmse"              # Evaluation metric used for early stopping
}

model = xgb.XGBRegressor(**xgb_params)

# Initialize K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
for id in ids:
  for model_name in models:
    for lag in lags:
      print(f"ID: {id}, Lag: {lag}")
      glucose_path = f"{file_path}/{id}_glucose_processed.csv"
      df_glucose_filtered = pd.read_csv(glucose_path)
      df_features = pd.read_csv(f'{file_path}/{id}_feature_engineered_data.csv')
      features, labels = extract_consecutive_entries_ending_at(
          df_glucose_filtered,
          df_features[final_features],
          time_column_main='Device Timestamp',
          time_column_minute='Time',
          num_entries=1440,
          lag=lag,
          fill_missing=False)

      results = perform_kfold_cv(features, labels, model, model_name, lag, id)
      final_data.append(results)

# Save all the plots to the PDF file
pdf_pages.close()

In [ ]:
# Convert to a DataFrame
df = pd.DataFrame(final_data)

# Save the DataFrame to a CSV file
csv_file = f"{results_path}/captured_metrics_{current_datetime}.csv"
df.to_csv(csv_file, index=False)

print(f"Metrics saved to {csv_file}")

In [ ]:
df_preds_vs_actuals = pd.DataFrame(preds_vs_actuals)

csv_file_preds_vs_actuals = f"{results_path}/captured_preds_vs_actuals_{current_datetime}.csv"

df_preds_vs_actuals.to_csv(csv_file_preds_vs_actuals, index=False)

print(f"Predictions vs. Actuals saved to {csv_file_preds_vs_actuals}")